# Example on using RDKit, clustering techniques and xTB to find relevant conformers

TODO Cite Rescoss


In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import logging
import sys

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import SVG
from rdkit import Chem
from rdkit.Chem import AllChem, PandasTools
from rdkit.Chem.Draw import MolsToGridImage, MolToImage, rdMolDraw2D

In [ ]:
try:
    import ppqm
except ModuleNotFoundError:
    import pathlib

    cwd = pathlib.Path().resolve().parent
    sys.path.append(str(cwd))
    import ppqm

In [ ]:
# Set logging
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger("ppqm").setLevel(logging.INFO)
logging.getLogger("xtb").setLevel(logging.INFO)

In [ ]:
# Set DataFrames visuals
PandasTools.RenderImagesInAllDataFrames(images=True)
pd.set_option('display.float_format','{:.2f}'.format)

## Define a Molecule

In [ ]:
smiles = "Cc1cc(NCCO)nc(-c2ccc(Br)cc2)n1"  # CHEMBL1956589

In [ ]:
molobj = Chem.MolFromSmiles(smiles)

In [ ]:
molobj

In [ ]:
# Generate a lot of conformers

In [ ]:
# using rdkit

In [ ]:
# Filter a lot of conformers to unique

In [ ]:
# explain traits

In [ ]:
# selecting clustering algorithm

In [ ]:
# Optimize conformers

In [ ]:
# Watch the changes to conformer shape space